# 📘 Galileo + OpenTelemetry Tracing Guide (Python)

This notebook demonstrates how to send OpenTelemetry traces to Galileo using OpenInference.

🔗 See full documentation [here](https://v2galileo.mintlify.app/guides/logs/otel)

In [ ]:
pip install -U \
    openinference-instrumentation-anthropic \
    opentelemetry-sdk \
    opentelemetry-exporter-otlp \
    opentelemetry-api \
    "opentelemetry-proto>=1.12.0" -q

In [ ]:
GALILEO_API_KEY = "YOUR_API_KEY"
PROJECT_NAME = "YOUR-PROJECT-NAME"
LOG_STREAM_NAME = "my_log_stream"



## 🔐 Authentication and Header Setup

In [ ]:
import os

headers = {
    "Galileo-API-Key": GALILEO_API_KEY,
    "project": PROJECT_NAME,
    "logstream": LOG_STREAM_NAME,
}
# Format headers as a comma-separated key=value string
# Example: Galileo-API-Key=abc123,project_name=my_project,log_stream_name=my_run
os.environ['OTEL_EXPORTER_OTLP_TRACES_HEADERS'] = ",".join([f"{k}={v}" for k, v in headers.items()])

## ⚙️ OpenTelemetry Setup

In [ ]:
from openinference.instrumentation.anthropic import AnthropicInstrumentor
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry import trace as trace_api
from opentelemetry.sdk.trace.export import BatchSpanProcessor, ConsoleSpanExporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

os.environ["ANTHROPIC_API_KEY"] = "YOUR_KEY_HERE"

# OTEL endpoint for Galileo
endpoint = "http://api.galileo.ai/otel/traces"

# Setup tracer provider
tracer_provider = trace_sdk.TracerProvider()
tracer_provider.add_span_processor(BatchSpanProcessor(OTLPSpanExporter(endpoint)))

# Optional: log locally
tracer_provider.add_span_processor(BatchSpanProcessor(ConsoleSpanExporter()))

# Register the provider
trace_api.set_tracer_provider(tracer_provider=tracer_provider)

# Antropic Instrument
AnthropicInstrumentor().instrument(tracer_provider=tracer_provider)

print("Antropic OTEL tracing initialized.")


OTEL tracing initialized.


In [ ]:
from anthropic import Anthropic

client = Anthropic()

response = client.messages.create(
    max_tokens=1024,
    messages=[
        {
            "role": "user",
            "content": "Tell me about the history of Iceland!",
        }
    ],
    model="claude-3-opus-20240229",
)